<a href="https://colab.research.google.com/github/alimomennasab/ASL-Translator/blob/main/CS4200_3DCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Data Preprocessing

In [ ]:
# if a label folder has under 5 videos, add to ignore list
import os

dataset_dir = "/content/drive/MyDrive/WLASL/WLASL100_train_augmented_60frames"
ignore_labels = []
for label in os.listdir(dataset_dir):
    label_dir = os.path.join(dataset_dir, label)
    if len(os.listdir(label_dir)) < 5:
        ignore_labels.append(label)

print(f"Ignored labels: {sorted(ignore_labels)}")

# Ignored labels: []

Ignored labels: []


In [4]:
# histogram of video frames

import cv2
import os
import matplotlib.pyplot as plt
import tqdm
import numpy as np

#vid_path = "/content/drive/MyDrive/WLASL/WLASL100_train_augmented/"
vid_path = "/content/drive/MyDrive/WLASL/WLASL100_train_augmented_40frames/"

frames = []
for folder in tqdm.tqdm(os.listdir(vid_path)):
  folder_path = os.path.join(vid_path, folder)
  for vid in os.listdir(folder_path):
    cap = cv2.VideoCapture(os.path.join(folder_path, vid))

    if not cap.isOpened():
        print("Error: Could not open video.")
    else:
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(f"Total number of frames: {total_frames}")
        frames.append((total_frames, vid))

        cap.release()

frame_counts = [f[0] for f in frames]

# Plot
plt.figure(figsize=(15, 6))
interval = 5
start = 15
end = 160
plt.hist(frame_counts, bins= int((end - start)//interval))
plt.xlabel('Number of Frames')
plt.ylabel('Frequency')
plt.title('Histogram of Training Set Video Frames')
plt.xlim(start, end)
plt.xticks(np.arange(start, end+1, interval))
plt.yticks(np.arange(0, 1000, 50))

plt.show()

# Stats
avg_frames = sum(frame_counts) / len(frame_counts)
min_tuple = min(frames, key=lambda x: x[0])
max_tuple = max(frames, key=lambda x: x[0])
print(f"Average number of frames: {avg_frames:.2f}")
print(f"Minimum frame count: {min_tuple[0]}, video={min_tuple[1]}")
print(f"Maximum frame count: {max_tuple[0]}, video={max_tuple[1]}")

# Average number of frames: 63.07
# Minimum frame count: 19, video=06334_AUG1_zoom_fast.mp4
# Maximum frame count: 155, video=63232_AUG2_bright_mirror.mp4

 24%|██▍       | 24/98 [01:57<06:01,  4.88s/it]


KeyboardInterrupt: 

Data Augmentation

In [ ]:
# Go through a dataset and transform every video to have a target number of frames

import cv2
import os
import tqdm
import numpy as np

# duplicate frames in a video starting with the middle frames and expanding outwards until we reach a target frame count
def duplicate_frames(frames, target):
    n = len(frames)
    if n == 0:
        print("Warning: Empty frames list")
        return [], "duplicated"

    indices = list(range(n))
    mid = n // 2
    left, right = mid - 1, mid + 1
    side = "left"

    while len(indices) < target:
        if side == "left" and left >= 0:
            indices.append(left)
            left -= 1
        elif side == "right" and right < n:
            indices.append(right)
            right += 1
        else:
            indices.append(mid)  # fallback
        side = "right" if side == "left" else "left"

    indices.sort()
    new_frames = [frames[i] for i in indices]
    return new_frames, "duplicated"


# remove frames alternately from start and end until we reach a target frame count
def remove_frames(frames, target):
    remove_from_start = True
    start = 0
    end = len(frames)

    while (end - start) > target:
        if remove_from_start:
            start += 1
        else:
            end -= 1
        remove_from_start = not remove_from_start

    return frames[start:end], "removed"


target_frames = 40
data_dir = "/content/drive/MyDrive/WLASL/WLASL100_test/"
output_dir = f"/content/drive/MyDrive/WLASL/WLASL100_test_{target_frames}frames/"
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")

for folder in tqdm.tqdm(os.listdir(data_dir)):
    print(f"\nProcessing folder {folder}")
    folder_path = os.path.join(data_dir, folder)
    output_folder_path = os.path.join(output_dir, folder)
    os.makedirs(output_folder_path, exist_ok=True)

    for vid in (os.listdir(folder_path)):
        frames = []
        suffix = ""

        cap = cv2.VideoCapture(os.path.join(folder_path, vid))
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        cap.release()

        if len(frames) < target_frames:
            # print(f"Video {vid} has less than 60 frames")
            frames, suffix = duplicate_frames(frames, target_frames)
        elif len(frames) > target_frames:
            # print(f"Video {vid} has more than 60 frames")
            frames, suffix = remove_frames(frames, target_frames)
        # else:
        #     print(f"Video {vid} already has {target_frames} frames")

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = 30
        if frames:
            height, width, _ = frames[0].shape

            # add suffix to filename
            base, extension = os.path.splitext(vid)
            if suffix:
                output_filename = f"{base}_{suffix}{extension}"
            else:
                output_filename = vid

            output_video_path = os.path.join(output_folder_path, output_filename)
            out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

            for frame in frames:
                out.write(frame)

            out.release()
        else:
            print(f"Warning: No frames were processed for video {vid} in folder {folder}. Skipping video writing.")


Output directory: /content/drive/MyDrive/WLASL/WLASL100_test_40frames/


  0%|          | 0/88 [00:00<?, ?it/s]


Processing folder shirt


  1%|          | 1/88 [00:03<04:29,  3.09s/it]


Processing folder help


...

 98%|█████████▊| 86/88 [02:06<00:02,  1.44s/it]


Processing folder paint


 99%|█████████▉| 87/88 [02:08<00:01,  1.59s/it]


Processing folder accident


100%|██████████| 88/88 [02:09<00:00,  1.48s/it]


Data loading

In [2]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import cv2
from pathlib import Path
from PIL import Image
import numpy as np


TRAIN_DATASET_DIR = "/content/drive/MyDrive/WLASL/WLASL100_train_augmented_40frames"
VAL_DATASET_DIR = '/content/drive/MyDrive/WLASL/WLASL100_val_40frames'
IMG_SIZE = 256
BATCH_SIZE = 4


if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU")
else:
    device = "cpu"
    print("Using CPU")

# Image transforms
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, class_map=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = [] # stores (video_path, label_idx) tuples

        if class_map is None:
            self.class_to_idx = {}
            self.idx_to_class = []
            label_folders = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
            for i, label_name in enumerate(label_folders):
                self.class_to_idx[label_name] = i
                self.idx_to_class.append(label_name)
        else:
            self.class_to_idx = class_map['class_to_idx']
            self.idx_to_class = class_map['idx_to_class']

        # Populate samples and create class mappings
        for label_name in os.listdir(root_dir):
            if label_name in self.class_to_idx: # Only include labels present in the class_to_idx map
                label_path = os.path.join(root_dir, label_name)
                if os.path.isdir(label_path):
                    label_idx = self.class_to_idx[label_name]
                    for video_file in os.listdir(label_path):
                        if video_file.lower().endswith('.mp4'):
                            video_path = os.path.join(label_path, video_file)
                            self.samples.append((video_path, label_idx))

        self.class_names = self.idx_to_class
        print(f"Found {len(self.samples)} video samples across {len(set(label for _, label in self.samples))} classes in {root_dir}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        frames = self._load_all_frames(video_path)

        # Apply transformations to each frame
        transformed_frames = []
        for frame in frames:
            pil_image = Image.fromarray(frame)
            if self.transform:
                transformed_frames.append(self.transform(pil_image))

        frames_tensor = torch.stack(transformed_frames) # Shape: (T, C, H, W)
        frames_tensor = frames_tensor.permute(1, 0, 2, 3) # Shape: (C, T, H, W)

        return frames_tensor, label

    def _load_all_frames(self, video_path):
      cap = cv2.VideoCapture(video_path)
      if not cap.isOpened():
          print(f"Error: Could not open video file: {video_path}")
          return []

      extracted_frames_rgb = []
      while True:
          ret, frame = cap.read()
          if not ret:
              break
          rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          extracted_frames_rgb.append(rgb_frame)

      cap.release()

      if len(extracted_frames_rgb) == 0:
          print(f"Warning: No frames extracted from video {video_path}")

      return extracted_frames_rgb


# Load dataset
print("Creating dataset")
train_ds = VideoDataset(TRAIN_DATASET_DIR, transform=transform)

# Use the class mapping from the training set for validation set
val_class_map = {'class_to_idx': train_ds.class_to_idx, 'idx_to_class': train_ds.idx_to_class}
val_ds = VideoDataset(VAL_DATASET_DIR, transform=transform, class_map=val_class_map)

# Dataloaders
print("Creating dataloaders")
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", class_names[:10], "...")
print("num_classes =", num_classes)


Using GPU
Creating dataset
Found 2845 video samples across 98 classes in /content/drive/MyDrive/WLASL/WLASL100_train_augmented_40frames
Found 97 video samples across 97 classes in /content/drive/MyDrive/WLASL/WLASL100_val_40frames
Creating dataloaders
Classes: ['accident', 'africa', 'all', 'apple', 'basketball', 'bed', 'before', 'bird', 'birthday', 'black'] ...
num_classes = 98


Model training

In [ ]:
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import json
import os
from torchvision.models.video import r3d_18
from tqdm import tqdm
from google.colab import drive

def plot_losses(train_losses: list, val_losses: list):
    epochs = range(1, len(train_losses) + 1)
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss Curves')
    plt.legend()
    plt.show()



# Load pretrained 3D CNN
num_classes = len(class_names)
model = r3d_18(weights="KINETICS400_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 8
start_epoch = 0

# Load saved model from checkpoint
drive.mount('/content/drive')
CHECKPOINT_PATH = "/content/drive/MyDrive/3d_cnn_asl_checkpoint.pt"
MODEL_PATH = "/content/drive/MyDrive/3d_cnn_asl.pt"
LABELS_PATH = "/content/drive/MyDrive/3d_cnn_asl_labels.json"

if os.path.exists(CHECKPOINT_PATH):
    print("Loading checkpoint...")
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    start_epoch = checkpoint["epoch"] + 1
    train_losses = checkpoint["train_losses"]
    val_losses = checkpoint["val_losses"]
    print(f"Resuming from epoch {start_epoch}")
else:
    train_losses, val_losses = [], []

# Training loop
for epoch in range(start_epoch, EPOCHS):
    model.train()
    epoch_loss, correct, total = 0, 0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, preds = outputs.max(1)
        total += labels.size(0)
        correct += preds.eq(labels).sum().item()
        pbar.set_postfix(loss=loss.item(), acc=100.*correct/total)

    train_losses.append(epoch_loss / len(train_loader))

    # Validation
    model.eval()
    val_epoch_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_epoch_loss += loss.item()
            _, preds = outputs.max(1)
            val_total += labels.size(0)
            val_correct += preds.eq(labels).sum().item()
    val_losses.append(val_epoch_loss / len(val_loader))
    val_acc = 100. * val_correct / val_total

    print(f"Epoch {epoch+1}: Train Acc={100.*correct/total:.2f}% | Val Acc={val_acc:.2f}%")

    # Save checkpoint every epoch
    checkpoint = {
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "train_losses": train_losses,
        "val_losses": val_losses
    }
    torch.save(checkpoint, CHECKPOINT_PATH)
    print(f"Checkpoint saved at epoch {epoch+1}")



torch.save(model.state_dict(), MODEL_PATH)
with open(LABELS_PATH, "w") as f:
    json.dump(class_names, f)
print("Final model saved to:", MODEL_PATH)

Downloading: "https://download.pytorch.org/models/r3d_18-b3b3357e.pth" to /root/.cache/torch/hub/checkpoints/r3d_18-b3b3357e.pth


100%|██████████| 127M/127M [00:01<00:00, 109MB/s]
Epoch 1/8: 100%|██████████| 712/712 [52:34<00:00,  4.43s/it, acc=8.89, loss=4.78]


Epoch 1: Train Acc=8.89% | Val Acc=4.12%


Epoch 2/8: 100%|██████████| 712/712 [52:50<00:00,  4.45s/it, acc=42.5, loss=4.76]


Epoch 2: Train Acc=42.53% | Val Acc=17.53%


Epoch 3/8:  79%|███████▊  | 559/712 [41:36<11:23,  4.47s/it, acc=83.9, loss=0.619]

Save Models/Labels

In [ ]:
import json
from google.colab import drive

drive.mount('/content/drive')

MODEL_PATH = "/content/drive/MyDrive/asl_overlay_resnet18.pt"
LABELS_PATH = "/content/drive/MyDrive/asl_overlay_labels.json"

torch.save(model.state_dict(), MODEL_PATH)

with open(LABELS_PATH, "w") as f:
    json.dump(class_names, f)

print("Model saved to:", MODEL_PATH)
print("Labels saved to:", LABELS_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model saved to: /content/drive/MyDrive/asl_overlay_resnet18.pt
Labels saved to: /content/drive/MyDrive/asl_overlay_labels.json


Testing model

In [ ]:
import cv2
import numpy as np
import torch
import json
import matplotlib.pyplot as plt
import tqdm
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


IMG_SIZE = 256
CROP_AMOUNT = 75 # pixels

# Test dataset augmentations
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])


def crop_border_frames(frames, crop_amount=75):
    h, w = frames[0].shape[:2]
    return [f[:, crop_amount:w - crop_amount] for f in frames]

# Test dataset that only uses words the model trained on
class TestDataset(Dataset):
    def __init__(self, root_dir, class_to_idx, transform=None, valid_labels=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.class_to_idx = class_to_idx
        self.valid_labels = set(valid_labels or [])
        self.samples = []

        for class_dir in sorted(self.root_dir.iterdir()):
            # only keep labels that were in training
            if not class_dir.is_dir() or class_dir.name not in self.valid_labels:
                continue
            for video_path in class_dir.glob("*.mp4"):
                self.samples.append((video_path, class_dir.name))

        print(f"Found {len(self.samples)} videos across {len(set(l for _, l in self.samples))} classes")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        frames = []
        cap = cv2.VideoCapture(str(video_path))
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)
        cap.release()

        # cropping
        frames = crop_border_frames(frames, crop_amount=CROP_AMOUNT)

        # stacking
        stacked_img = stack_frames_max(frames)
        img = Image.fromarray(cv2.cvtColor(stacked_img, cv2.COLOR_BGR2RGB))
        if self.transform:
            img = self.transform(img)
        return img, self.class_to_idx[label]


# Config
FRAME_SKIP = 2
ignored_labels = []
test_dir = "/content/drive/MyDrive/WLASL/WLASL100_test"
LABELS_PATH = "/content/drive/MyDrive/asl_overlay_labels.json"
MODEL_PATH = "/content/drive/MyDrive/asl_overlay_resnet18.pt"

# Build labels list
with open(LABELS_PATH, "r") as f:
    class_names = json.load(f)
class_to_idx = {c: i for i, c in enumerate(class_names)}
train_labels = set(class_names)
test_labels = {d.name for d in Path(test_dir).iterdir() if d.is_dir()}
in_train_and_test = sorted(list(train_labels & test_labels - set(ignored_labels)))
print(f"Testing on {len(in_train_and_test)} shared classes between training & testing")

# Initialize dataset, dataloader, and model
test_dataset = TestDataset(test_dir, class_to_idx, transform=transform, valid_labels=in_train_and_test)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

model = models.resnet50(weights=None)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model = model.to(device)
model.eval()

# Testing
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in tqdm.tqdm(test_loader, desc="Testing"):
        images = images.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute & display metrics
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# cm = confusion_matrix(all_labels, all_preds)
# print("Confusion Matrix:")
# print(cm)

# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
# disp.plot(xticks_rotation='vertical', cmap='Blues')
# plt.title("Confusion Matrix")
# plt.show()

Testing on 88 shared classes between training & testing
Found 98 videos across 88 classes


Testing: 100%|██████████| 4/4 [00:56<00:00, 14.21s/it]

Test Accuracy: 24.49%
